# Traitement des données de financements 

In [3]:
#importer les packages
import pandas as pd
import numpy as np
from code_utils.utils import extract_first_name,replace_all,get_scanR_structure
from code_utils.cached_data_handler import get_structure, get_person
from code_utils.features_into_dictionnary import persons, projects, address
from code_utils.id_from_orcid import orcid_to_idref
from tqdm import tqdm
import pprint as pp
tqdm.pandas()
import requests
from code_utils.pickle import load_cache,write_cache
import os
from dotenv import load_dotenv

load_dotenv()

Authorization = os.getenv('Authorization_access_185')
Authorization_ORCID = os.getenv('Authorization_cluster_BSO_ORCID')
url_cluster = os.getenv('url_cluster')

In [4]:
sources=pd.read_json('sources.json')

In [5]:
source=list(sources.keys())[0]
source

'ANR'

In [5]:
#cache structures, personnes et orcid avec differentes sources de donnees
cached_data = {}
try:
    cached_data = load_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
except:
    write_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
    
cached_data_persons = {}
try:
    cached_data_persons = load_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
except:
    write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
    
cached_data_orcid = {}
try:
    cached_data_orcid = load_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")
except:
    write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")

# ENVOI DES PROJETS SUR SCANR

In [30]:
if len(sources[source]['identifiants_preferes_personne'])==2:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partenaires_id_personne_ORCID.json")
elif len(sources[source]['identifiants_preferes_personne'])==1:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partenaires_id_personne.json")
else:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partenaires_id_structures.json")

In [65]:
df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure']=df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure'].apply(lambda y: y[0])

In [68]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_person']=df_partenaires.apply(lambda row: identifiant_prefere(row,sources[source]['identifiants_preferes_personne']), axis=1)
    df_partenaires['persons']=df_partenaires.progress_apply(lambda row: persons(row,sources[source]['prenom'],sources[source]['nom']) ,axis=1)
else:
    df_partenaires['persons']=np.nan

100%|██████████| 72089/72089 [00:03<00:00, 20437.28it/s]


In [70]:
if source != 'SIRANO':
    df_partenaires=df_partenaires.groupby([sources[source]['code_projet']]).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False).reset_index()
else:
    df_projets=df_partenaires.groupby([sources[source]['code_projet'], sources[source]['annee'], sources[source]['acronyme'],sources[source]['titre'],sources[source]['budget']], dropna=False).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False)

In [71]:
# amener les projets depuis le site 
if source=='ANR':
    page_projets_10 = requests.get(sources[source]['url_projets']).json()
    colonnes_projets_10 = page_projets_10['columns']
    donnees_projets_10 = page_projets_10['data']
    df_projets=pd.DataFrame(data=donnees_projets_10,columns=colonnes_projets_10)
elif source=='IRESP':
    df_projets1=pd.read_csv(sources[source]['url_projets1'] ,sep=";", encoding='UTF-8')
    df_projets2=pd.read_csv(sources[source]['url_projets2'] ,sep=";", encoding='UTF-8')
    df_projets=pd.concat([df_projets1,df_projets2])
    df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet_FR']=df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet']
elif source!='SIRANO':
    df_projets=pd.read_csv(sources[source]['url_projets'] ,sep=";", encoding='ISO-8859-1')

df_projets=df_projets.reset_index()
del df_projets['index']

In [72]:
if source!='SIRANO':
    df_projets=pd.merge(df_projets,df_partenaires,on=sources[source]['code_projet'], how='left')
else :
    df_projets['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[sources[source]['annee']]}-{row[sources[source]['acronyme']]}" , axis=1)
    del df_projets['code_projet']
    sources[source]['code_projet']='id'

df_projets['type']=source
df_projets['name']=df_projets.progress_apply(lambda row: projects(row,sources[source]['titre_fr'],sources[source]['titre_en']) ,axis=1)
df_projets['description']=df_projets.progress_apply(lambda row: projects(row,sources[source]['resume_fr'],sources[source]['resume_en']) ,axis=1)
df_projets.loc[:,sources[source]['budget']]=df_projets.loc[:,sources[source]['budget']].apply(lambda x : float(str(x).replace('.0','').replace('.00','').replace(' ','').replace(',','.').replace('€','')))
df_projets=df_projets.rename(columns={sources[source]['annee']:'year',sources[source]['acronyme']:'acronym',
                                      sources[source]['budget']:'budget_financed',sources[source]['code_projet']:'id'})
df_projets=df_projets[['id','type','name','description','acronym','year','budget_financed','persons']]

100%|██████████| 22628/22628 [00:00<00:00, 24608.27it/s]


In [75]:
df_projets

,id,type,name,description,acronym,year,budget_financed,persons
0,ANR-24-SRSE-0016,ANR,{'fr': 'Explorer l'interface neurovasculaire d...,{'fr': 'Le vaste réseau de nerfs périphériques...,MINerVA,2024,15457.50,"[{'first_name': 'Isabelle', 'last_name': 'BRUN..."
1,ANR-24-SRSE-0015,ANR,{'fr': 'Avancer l'innovation digitale démocrat...,{'fr': 'The Advancing Digital Democratic Innov...,ADDI,2024,16052.90,"[{'id': 'idref224400711', 'first_name': 'Umber..."
2,ANR-24-SRSE-0014,ANR,{'fr': 'Plate-forme multi-maladies vectorielle...,{'fr': 'Il est indispensable de déterminer la ...,IMPACTING,2024,16999.82,"[{'id': 'idref253652642', 'first_name': 'Antoi..."
3,ANR-24-SRSE-0013,ANR,{'fr': 'Innovations axées sur le marché pour l...,{'fr': 'Les emballages plastiques constituent ...,MERKATO,2024,169974.00,"[{'first_name': 'Bastien', 'last_name': 'ROCA'..."
4,ANR-24-SRSE-0012,ANR,{'fr': 'Rendre les moustiques intolérants aux ...,{'fr': 'L'incidence mondiale des maladies caus...,inTOLERABLE,2024,15686.50,"[{'id': 'idref176597514', 'first_name': 'Maria..."
...,...,...,...,...,...,...,...,...
22623,ANR-10-AALI-0005,ANR,None,None,STIMULATE,2010,196347.00,"[{'role': 'scientific-officer###491282364'}, {..."
22624,ANR-10-AALI-0004,ANR,None,None,NACODEAL,2010,196381.50,[{'role': 'scientific-officer###491282364'}]
22625,ANR-10-AALI-0003,ANR,None,None,LILY,2010,379160.00,"[{'role': 'scientific-officer###512338633'}, {..."
22626,ANR-10-AALI-0002,ANR,None,None,HOST,2010,1033012.93,[{'role': 'scientific-officer###22690001700014...


In [74]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

{'id': 'ANR-24-SRSE-0016',
 'type': 'ANR',
 'name': {'fr': "Explorer l'interface neurovasculaire des nerfs périphériques dans l'homéostasie, la pathologie et la réparation",
  'en': 'Unlocking the neurovascular interface of peripheral nerves in homeostasis, disease and repair'},
 'description': {'fr': 'Le vaste réseau de nerfs périphériques se développe simultanément et en contact étroit avec le système vasculaire. Cette relation spatiale et fonctionnelle persiste à l\'âge adulte, formant une interface neurovasculaire (NV) qui protège le tissu nerveux vulnérable par le biais d\'une barrière hémato-nerveuse (BNB), semblable à la barrière hémato-encéphalique (BBB). Malgré son rôle essentiel dans la santé du système nerveux périphérique (SNP), notre compréhension de l\'interface NV est limitée, ce qui a un impact sur la conception de thérapies pour les troubles du SNP liés à un dysfonctionnement vasculaire. Notre proposition interdisciplinaire vise à définir les caractéristiques moléculai

In [ ]:
#envoi
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
pd.Series([x.get('issues').get('id') for x in err]).drop_duplicates().tolist() 

pour mettre à jour

In [76]:
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANR"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={"type":"ANR"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_projets['id']) if x not in list_ids]

projets_a_retirer=[x for x in list_ids if x not in list(df_projets['id'])]

df_projets = df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)]

page 1
k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183

In [80]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

{'id': 'ANR-24-SRSE-0016',
 'type': 'ANR',
 'name': {'fr': "Explorer l'interface neurovasculaire des nerfs périphériques dans l'homéostasie, la pathologie et la réparation",
  'en': 'Unlocking the neurovascular interface of peripheral nerves in homeostasis, disease and repair'},
 'description': {'fr': 'Le vaste réseau de nerfs périphériques se développe simultanément et en contact étroit avec le système vasculaire. Cette relation spatiale et fonctionnelle persiste à l\'âge adulte, formant une interface neurovasculaire (NV) qui protège le tissu nerveux vulnérable par le biais d\'une barrière hémato-nerveuse (BNB), semblable à la barrière hémato-encéphalique (BBB). Malgré son rôle essentiel dans la santé du système nerveux périphérique (SNP), notre compréhension de l\'interface NV est limitée, ce qui a un impact sur la conception de thérapies pour les troubles du SNP liés à un dysfonctionnement vasculaire. Notre proposition interdisciplinaire vise à définir les caractéristiques moléculai

In [81]:
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [79]:
len(projets_a_ajouter)

872

Modifications

In [148]:
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANR"}&projection={"id":1,"year":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])
nbr_page

46

In [154]:
list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={%22type%22:%22ANR%22}&projection={%22id%22:1,%22year%22:1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        list_ids.append(page['data'][k]['year'])

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30
page 31
page 32
page 33
page 34
page 35
page 36
page 37
page 38
page 39
page 40
page 41
page 42
page 43
page 44
page 45
page 46


In [155]:
max(list_ids)

2024

In [ ]:
df_projets = df_projets[df_projets['id'].apply(lambda x: x in list_ids)]

In [ ]:
"""
en principe, il faut que je je mette à jour tous les projets 
qui ne sont pas clos avec les titres résumés et persons, mais 
peut etre qu'on peut mettre à jour automatiquement les projets
des 3 dernières années ? 

"""

err=[]
for id in df_projets.iterrows() :
    url = f"http://185.161.45.213/projects/projects/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"year": 2023}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)
